In [4]:
import numpy as np
import pandas as pd

# data = pd.read_pickle("rawData.df")
data = pd.read_pickle("NEWDATA/newData.df")
# HAVE TO MAKE SURE AS MUCH OF THE DATASET IS KEPT TIL HERE AS POSSIBLE FOR IMPUTATION


# https://towardsdatascience.com/stop-using-mean-to-fill-missing-data-678c0d396e22

In [5]:
# Deal with length values
def parse_ht(ht):
    # format: 7' 0.0"
    ht_ = ht.split("' ")
    ft_ = float(ht_[0])
    in_ = float(ht_[1].replace("\'\'",""))
    return (12*ft_) + in_

data["HEIGHT"] = [parse_ht(val) if type(val)==str else val for val in data["HEIGHT"]]
data["HEIGHT_W_SHOES"] = [parse_ht(val) if type(val)==str else val for val in data["HEIGHT_W_SHOES"]]
data["REACH"] = [parse_ht(val) if type(val)==str else val for val in data["REACH"]]
data["WINGSPAN"] = [parse_ht(val) if type(val)==str else val for val in data["WINGSPAN"]]

In [6]:
# Reformat weird columns
data["BODY_FAT"] = [float(val.replace("%", ""))/100 if type(val)!=float else val for val in data["BODY_FAT"]]
data["RSCI"] = [int(str(val).replace("T", "")) if str(val)!="nan" else np.nan for val in data["RSCI"]]
data["BENCH_PRESS"] = [int(val) if str(val)!="nan" else val for val in data["BENCH_PRESS"]]

if "age" in data.columns:
    data["age"] = [int(val) if str(val)!="nan" else val for val in data["age"]]
    
# Switch some datatypes
datatypes = {"SHUTTLE_RUN":float, "MAX_VERTICAL":float, "WEIGHT":float, 
             "THREE_QUARTER_SPRINT":float, 
             "STANDING_VERTICAL":float, "m1":float, "m2":float, "m3":float, "m4":float, "m5":float, "m6":float, 
             "RSCI":float, "MP_per_PF":float, "MP_per_3PA":float}
data = data.astype(datatypes)

In [7]:
# Add a variable that says whether they actually did the combine
data["didCombine"] = True
for i in range(len(data)):
    entry = data.iloc[i]
    if pd.isnull(entry["STANDING_VERTICAL"]) and pd.isnull(entry["SHUTTLE_RUN"]) and pd.isnull(entry["MAX_VERTICAL"]) and pd.isnull(entry["BENCH_PRESS"]):
        data["didCombine"].iloc[i] = False


/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
# Fill in the missing values using mean imputation
# impute_mean_cols = ["PER", "ORtg", "DRtg", "SHUTTLE_RUN", "THREE_QUARTER_SPRINT", "STANDING_VERTICAL", 
#                     "MAX_VERTICAL", "BENCH_PRESS", "BODY_FAT", "HAND_LENGTH", "HAND_WIDTH", "HEIGHT",
#                     "HEIGHT_W_SHOES", "REACH", "WEIGHT", "WINGSPAN", "SOS", "OWS", "DWS", 
#                     "eFG%", "ORB%", "DRB%", "AST%", "TOV%", "STL%", "BLK%", "USG%", #"age",
#                    ]

# for col in impute_mean_cols:
#     data[col] = data.fillna(np.mean(data.loc[data[col].isnull()==False]))[col]

In [9]:
# Fill in missing values using MICE regression
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
impute_cols = ["PER", "ORtg", "DRtg", "SHUTTLE_RUN", "THREE_QUARTER_SPRINT", "STANDING_VERTICAL", 
               "MAX_VERTICAL", "BENCH_PRESS", "BODY_FAT", "HAND_LENGTH", "HAND_WIDTH", "HEIGHT",
               "HEIGHT_W_SHOES", "REACH", "WEIGHT", "WINGSPAN", "SOS", "OWS", "DWS", 
               "eFG%", "ORB%", "DRB%", "AST%", "TOV%", "STL%", "BLK%", "USG%", #"age",
               ]

imputer = IterativeImputer(max_iter=100)
data[impute_cols] = imputer.fit_transform(data[impute_cols])

In [10]:
# Save the impute device
import pickle
with open('imputer.pkl', 'wb') as f:
    pickle.dump(imputer, f)

In [7]:
# Constant fill some columns with max values
fill_max_cols = ["m"+str(i+1) for i in range(6)] + ["RSCI", "MP_per_PF", "MP_per_3PA"]
for col in fill_max_cols:
    data[col] = data.fillna(np.max(data[col]))[col]
    
# Constant fill some columns with min values (0 here)
fill_zero_cols = ["3P%", "awards"]
for col in fill_zero_cols:
    data[col] = data.fillna(0)[col]
    
data["WING_DIFF"] = [data["WINGSPAN"].iloc[i]-data["HEIGHT"].iloc[i] if pd.isnull(data["WING_DIFF"].iloc[i]) else data["WING_DIFF"].iloc[i] for i in range(len(data))]

In [8]:
# Show the remaining missing values
for col in data.columns:
    print(col, data[col].isnull().sum())

Pk 0
Tm 0
Player 0
Year 0
YEAR 375
POS 375
HEIGHT 0
HEIGHT_W_SHOES 0
REACH 0
WEIGHT 0
WINGSPAN 0
WING_DIFF 0
SHUTTLE_RUN 0
THREE_QUARTER_SPRINT 0
STANDING_VERTICAL 0
MAX_VERTICAL 0
BENCH_PRESS 0
BODY_FAT 0
HAND_LENGTH 0
HAND_WIDTH 0
collegeYear 375
gamesPlayed 379
minutes 379
FT% 379
3P% 0
SOS 0
PER 0
eFG% 0
ORB% 0
DRB% 0
AST% 0
TOV% 0
STL% 0
BLK% 0
USG% 0
OWS 0
DWS 0
FTA 375
FGA 375
MP 375
3PA 375
PTS 375
PF 375
AST 375
TOV 375
ORtg 0
DRtg 0
MP_per_PF 0
FTA_per_FGA 375
MP_per_3PA 0
PTS_per_FGA 375
AST_per_TOV 375
PPG 375
PPM 375
awards 0
RSCI 0
m1 0
m2 0
m3 0
m4 0
m5 0
m6 0
EWA 189
WS 171
WP 4
did3years 0
didAnyYears 0
didCombine 0


In [9]:
# One hot encode the catagory variables
data = pd.concat([data, pd.get_dummies(data["POS"])], axis=1)
data.drop(["POS", "YEAR"], axis=1, inplace=True)
# Drop the obvious columns that dont have games played - need to be at end to prevent bias
saveData = data[pd.isnull(data["gamesPlayed"])==False]
saveData.to_pickle("final_data1.df")

In [10]:
saveData["PPG"]

0       23.160000
1       14.555556
2       16.827586
3       16.342857
4       16.694444
          ...    
1056    12.500000
1060    11.937500
1061    12.103448
1064    15.580645
1066    14.615385
Name: PPG, Length: 691, dtype: float64

In [11]:
data['WING_DIFF']

0       0.000000
1       3.039779
2       6.138098
3       7.484422
4       4.500000
          ...   
1065    5.500000
1066    4.000000
1067    1.250000
1068    4.867940
1069    4.500000
Name: WING_DIFF, Length: 1070, dtype: float64

In [12]:
data.columns

Index(['Pk', 'Tm', 'Player', 'Year', 'HEIGHT', 'HEIGHT_W_SHOES', 'REACH',
       'WEIGHT', 'WINGSPAN', 'WING_DIFF', 'SHUTTLE_RUN',
       'THREE_QUARTER_SPRINT', 'STANDING_VERTICAL', 'MAX_VERTICAL',
       'BENCH_PRESS', 'BODY_FAT', 'HAND_LENGTH', 'HAND_WIDTH', 'collegeYear',
       'gamesPlayed', 'minutes', 'FT%', '3P%', 'SOS', 'PER', 'eFG%', 'ORB%',
       'DRB%', 'AST%', 'TOV%', 'STL%', 'BLK%', 'USG%', 'OWS', 'DWS', 'FTA',
       'FGA', 'MP', '3PA', 'PTS', 'PF', 'AST', 'TOV', 'ORtg', 'DRtg',
       'MP_per_PF', 'FTA_per_FGA', 'MP_per_3PA', 'PTS_per_FGA', 'AST_per_TOV',
       'PPG', 'PPM', 'awards', 'RSCI', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6',
       'EWA', 'WS', 'WP', 'did3years', 'didAnyYears', 'didCombine', 'C', 'F',
       'G'],
      dtype='object')